In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load the CSVs (no header)
test = pd.read_csv('/content/drive/MyDrive/test.csv', header=None)
train = pd.read_csv('/content/drive/MyDrive/train.csv', header=None)


# Take only the first N rows (e.g., 5000 for training, 1000 for testing)
train = train.sample(n=5000, random_state=42)
test = test.sample(n=1000, random_state=42)

# Combine text columns and rename
train.columns = ['sentiment', 'part1', 'part2']
train['review'] = train['part1'].astype(str) + " " + train['part2'].astype(str)
train = train[['review', 'sentiment']]  # Keep only necessary columns

test.columns = ['sentiment', 'part1', 'part2']
test['review'] = test['part1'].astype(str) + " " + test['part2'].astype(str)
test = test[['review', 'sentiment']]


In [ ]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')])
    return text


In [ ]:
import re
train['clean_review'] = train['review'].apply(preprocess_text)
test['clean_review'] = test['review'].apply(preprocess_text)


In [ ]:
# Vectorize the clean review text
tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train['clean_review'])
X_test = tfidf.transform(test['clean_review'])

y_train = train['sentiment']
y_test = test['sentiment']
train['sentiment'] = train['sentiment'].map({2: 1, 1: 0})
test['sentiment'] = test['sentiment'].map({2: 1, 1: 0})


In [ ]:
# Logistic Regression for binary classification
model = LogisticRegression()

# Cross-validation to avoid overfitting
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print("Average Cross-Validation Accuracy:", np.mean(cv_scores))


Average Cross-Validation Accuracy: 0.8352


In [ ]:
# Fit model
model.fit(X_train, y_train)

# Predict on test data
test['predicted_sentiment'] = model.predict(X_test)

# Evaluate (if test has labels)
print(classification_report(y_test, test['predicted_sentiment']))
print("Accuracy :",accuracy_score(y_test,test['predicted_sentiment']))

print("Confusion matrix :",confusion_matrix(y_test,test['predicted_sentiment']))


              precision    recall  f1-score   support

           0       0.85      0.86      0.86       496
           1       0.86      0.85      0.86       504

    accuracy                           0.86      1000
   macro avg       0.86      0.86      0.86      1000
weighted avg       0.86      0.86      0.86      1000

Accuracy : 0.857
Confusion matrix : [[429  67]
 [ 76 428]]
